# Evaluate Diversity by Distinct-n method

## Hypothesis
The Distinct-n score will increase with the dynamic pieces since the conversations are more diverse.

## Distinct-n


In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Change to root direction
root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
os.chdir(root_dir)

### Implementation of EAD

In [ ]:
from dataset.experiments.diversity_with_without_dynamic_pieces.Distinct_n_score.distinct_n import distinct_n_sentence_level
from dataset.experiments.diversity_with_without_dynamic_pieces.Distinct_n_score.distinct_n import distinct_n_corpus_level

# Example usage
conversations = [
    ["Hello, how are you?", "I am fine, thank you.", "And you?", "I'm good too."],
    ["What are you doing?", "Just reading a book.", "Which one?", "A mystery novel."],
    # More conversations...
]

ead_metric = distinct_n_corpus_level(conversations, 2)  # For bigrams
print("Distinct-n Metric:", ead_metric)


### Create Conversations with and without dynamic pieces


In [ ]:
import sys

sys.path.append(str(root_dir))

import json
import pandas as pd

# Local Imports
from dataset.chains.extraction_conversations import ExtractionConversationChain
from utils.start_langsmith_tracing import start_langsmith_tracing

start_langsmith_tracing(project_name="Experiment_EAD_Score")
user_profiles = []
user_profiles_dir = 'dataset/userprofiles/train_user_profiles.jsonl'
with open(user_profiles_dir, 'r') as file:
    for line in file:
        json_object = json.loads(line.strip())
        user_profiles.append(json_object)

user_profiles = user_profiles[:4] # Take only user

In [ ]:
import pickle

# Helper Function
def convert_conv_to_list_of_strings(conversation):
    """
    converts the returned conversation to a list of strings of the turns
    """
    conv = json.loads(conversation.content)['conversation']
    conversation_list_of_strings = []
    for turn in conv:
        conversation_list_of_strings.append(str(list(turn.values())[0]))
        
    return conversation_list_of_strings

In [ ]:
extraction_conversation_chain = ExtractionConversationChain()

all_distinct_n_scores_dynamic = []
all_distinct_n_scores_not_dynamic = []

for idx, user_profile in enumerate(user_profiles):
    user_profile_df = pd.DataFrame(user_profile["user_profile"])
    user_profile_df = user_profile_df.iloc[idx*2]

    print("========= NEW USER =========")
    print(user_profile_df)
    print("==")
    distinct_n_scores_dynamic = []
    distinct_n_scores_not_dynamic = []
    
    topic = user_profile_df["Main Category"] + "; " + user_profile_df["Subcategory"]
    user_preference = user_profile_df["Main Category"] + "; " + user_profile_df["Subcategory"] + "; " + user_profile_df["Detail Category"] + "; " + user_profile_df["Attributes"]
    
    
    if os.path.isfile(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/conversations_dynamic_{idx}_v3.pkl"):
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/conversations_dynamic_{idx}_v3.pkl", 'rb') as file:
            conversations_dynamic = pickle.load(file)
            print("load existing")
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/conversations_not_dynamic_{idx}_v3.pkl", 'rb') as file:
            conversations_not_dynamic = pickle.load(file)
            print("load existing")
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/metadatas_dynamic_{idx}_v3.pkl", 'rb') as file:
            metadatas_dynamic = pickle.load(file)
            print("load existing")
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/metadatas_not_dynamic_{idx}_v3.pkl", 'rb') as file:
            metadatas_not_dynamic = pickle.load(file)      
            print("load existing")
    else:
        # With dynamic pieces
        conversations_dynamic = []
        conversations_not_dynamic = []
        metadatas_dynamic = []
        metadatas_not_dynamic = []
        for i in range(10):
            if i == 0: # so that first sampled dynamic pieces are same to the ones in fixed dynamic pieces
                conversation_dynamic, metadata_dynamic = extraction_conversation_chain.generate_one_conversation(topic=topic, user_preference=user_preference, random_seed=idx)
                print("==== Metadata Dynamic =====")
                print(metadata_dynamic)
                print("===========================")
                conv = convert_conv_to_list_of_strings(conversation_dynamic)
                conversations_dynamic.append(conv)
                metadatas_dynamic.append(metadata_dynamic)
            else:
                conversation_dynamic, metadata_dynamic = extraction_conversation_chain.generate_one_conversation(topic=topic, user_preference=user_preference, random_seed = None)
                print("==== Metadata Dynamic =====")
                print(metadata_dynamic)
                print("===========================")                
                conv = convert_conv_to_list_of_strings(conversation_dynamic)
                conversations_dynamic.append(conv)
                metadatas_dynamic.append(metadata_dynamic)

        # Without dynamic pieces
            conversation_not_dynamic, metadata_not_dynamic = extraction_conversation_chain.generate_one_conversation(topic=topic, user_preference=user_preference, random_seed=idx)
            print("==== Metadata Not Dynamic =====")
            print(metadata_not_dynamic)
            print("===========================")            
            conv = convert_conv_to_list_of_strings(conversation_not_dynamic)
            conversations_not_dynamic.append(conv)
            metadatas_not_dynamic.append(metadata_not_dynamic)


        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/conversations_dynamic_{idx}_v3.pkl", 'wb') as file:
            pickle.dump(conversations_dynamic, file)
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/conversations_not_dynamic_{idx}_v3.pkl", 'wb') as file:
            pickle.dump(conversations_not_dynamic, file)
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/metadatas_dynamic_{idx}_v3.pkl", 'wb') as file:
            pickle.dump(metadatas_dynamic, file)
        with open(f"dataset/experiments/diversity_with_without_dynamic_pieces/Distinct_n_score/pickle/metadatas_not_dynamic_{idx}_v3.pkl", 'wb') as file:
            pickle.dump(metadatas_not_dynamic, file)   

    

    for i in range(len(conversations_dynamic)):
        flattened_conversations_dynamic = [item.split() for sublist in conversations_dynamic[:i+1] for item in sublist]
    
        current_distinct_n_scores_dynamic = []
        flattened_conversations_dynamic = [item for sublist in flattened_conversations_dynamic for item in sublist]
        print("flattened_conversation_dynamic")
        print(flattened_conversations_dynamic)
        ead_metric_dynamic_1 = distinct_n_sentence_level(flattened_conversations_dynamic, 1)  # For bigrams
        print("Distinct-1 Dynamic Metric:", ead_metric_dynamic_1)
        current_distinct_n_scores_dynamic.append(ead_metric_dynamic_1)
        ead_metric_dynamic_2 = distinct_n_sentence_level(flattened_conversations_dynamic, 2)  # For bigrams
        print("Distinct-2 Dynamic Metric:", ead_metric_dynamic_2)
        current_distinct_n_scores_dynamic.append(ead_metric_dynamic_2)
        ead_metric_dynamic_3 = distinct_n_sentence_level(flattened_conversations_dynamic, 3)  # For bigrams
        print("Distinct-3 Dynamic Metric:", ead_metric_dynamic_3)
        current_distinct_n_scores_dynamic.append(ead_metric_dynamic_3)

        distinct_n_scores_dynamic.append(current_distinct_n_scores_dynamic)

    all_distinct_n_scores_dynamic.append(distinct_n_scores_dynamic)
    

    for i in range(len(conversations_not_dynamic)): 
        flattened_conversations_not_dynamic = [item.split() for sublist in conversations_not_dynamic[:i+1] for item in sublist]
    
        current_distinct_n_scores_not_dynamic = []
        flattened_conversations_not_dynamic = [item for sublist in flattened_conversations_not_dynamic for item in sublist]
        print("flattened_conversation_not_dynamic")
        print(flattened_conversations_not_dynamic)
        ead_metric_not_dynamic_1 = distinct_n_sentence_level(flattened_conversations_not_dynamic, 1)  # For bigrams
        print("Distinct-1 Dynamic Metric:", ead_metric_not_dynamic_1)
        current_distinct_n_scores_not_dynamic.append(ead_metric_not_dynamic_1)
        ead_metric_not_dynamic_2 = distinct_n_sentence_level(flattened_conversations_not_dynamic, 2)  # For bigrams
        print("Distinct-2 Dynamic Metric:", ead_metric_not_dynamic_2)
        current_distinct_n_scores_not_dynamic.append(ead_metric_not_dynamic_2)
        ead_metric_not_dynamic_3 = distinct_n_sentence_level(flattened_conversations_not_dynamic, 3)  # For bigrams
        print("Distinct-3 Dynamic Metric:", ead_metric_not_dynamic_3)
        current_distinct_n_scores_not_dynamic.append(ead_metric_not_dynamic_3)

        distinct_n_scores_not_dynamic.append(current_distinct_n_scores_not_dynamic)

    all_distinct_n_scores_not_dynamic.append(distinct_n_scores_not_dynamic)
    print("==============================")

In [ ]:
all_distinct_n_scores_dynamic
# len(conversations_dynamic)-1
# conversations_dynamic

Distinct-1 Dynamic Metric: 0.45869947275922673
Distinct-1 Not Dynamic Metric: 0.4305343511450382
Distinct-2 Dynamic Metric: 0.7803163444639719
Distinct-2 Not Dynamic Metric: 0.749618320610687
Distinct-3 Dynamic Metric: 0.8822495606326889
Distinct-3 Not Dynamic Metric: 0.8473282442748091
Distinct-1 Dynamic Metric: 0.47771836007130125
Distinct-1 Not Dynamic Metric: 0.42835365853658536
Distinct-2 Dynamic Metric: 0.8110516934046346
Distinct-2 Not Dynamic Metric: 0.7591463414634146
Distinct-3 Dynamic Metric: 0.9162210338680927
Distinct-3 Not Dynamic Metric: 0.8551829268292683
Distinct-1 Dynamic Metric: 0.47766323024054985
Distinct-1 Not Dynamic Metric: 0.458528951486698
Distinct-2 Dynamic Metric: 0.8384879725085911
Distinct-2 Not Dynamic Metric: 0.8184663536776213
Distinct-3 Dynamic Metric: 0.9312714776632303
Distinct-3 Not Dynamic Metric: 0.9061032863849765
Distinct-1 Dynamic Metric: 0.4790697674418605
Distinct-1 Not Dynamic Metric: 0.444794952681388
Distinct-2 Dynamic Metric: 0.7984496124031008
Distinct-2 Not Dynamic Metric: 0.750788643533123
Distinct-3 Dynamic Metric: 0.8945736434108527
Distinct-3 Not Dynamic Metric: 0.8659305993690851

### Quantitative Results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

print(all_distinct_n_scores_dynamic)
print(all_distinct_n_scores_not_dynamic)
df_dynamic = pd.DataFrame(all_distinct_n_scores_dynamic)
df_not_dynamic = pd.DataFrame(all_distinct_n_scores_not_dynamic)

# Flatten the list of lists
flattened_data = [sum(sublist, []) for sublist in all_distinct_n_scores_dynamic]
# Assuming that each sublist has the same length, get the number of columns
num_columns = len(flattened_data[0])

# Create a DataFrame, where each column corresponds to the position in the sublists
df = pd.DataFrame(flattened_data, columns=[f'Distinct-{(i % 3) + 1}' for i in range(num_columns)])
print(df)
# Now calculate the mean for each column
means_dynamic = df.mean()
print("Means Dynamic: ", means_dynamic)

# Flatten the list of lists
flattened_data = [sum(sublist, []) for sublist in all_distinct_n_scores_not_dynamic]
# Assuming that each sublist has the same length, get the number of columns
num_columns = len(flattened_data[0])

# Create a DataFrame, where each column corresponds to the position in the sublists
df = pd.DataFrame(flattened_data, columns=[f'Distinct-{(i % 3) + 1}' for i in range(num_columns)])
print(df)
# Now calculate the mean for each column
means_not_dynamic = df.mean()
print("Means not dynamic: ", means_not_dynamic)

y_dynamic_1 = means_dynamic[::3]
y_dynamic_2 = means_dynamic[1::3]
y_dynamic_3 = means_dynamic[2::3]

y_not_dynamic_1 = means_not_dynamic[::3]
y_not_dynamic_2 = means_not_dynamic[1::3]
y_not_dynamic_3 = means_not_dynamic[2::3]

x = range(1, 11)

# Create subplots
fig, ax = plt.subplots(1, 3, figsize=(18, 6))  # Adjust figsize as needed

# Plot data with labels
ax[0].plot(x, y_dynamic_1, label='Dynamic Inputs')
ax[0].plot(x, y_not_dynamic_1, label='Fixed Inputs')
ax[1].plot(x, y_dynamic_2, label='Dynamic Inputs')
ax[1].plot(x, y_not_dynamic_2, label='Fixed Inputs')
ax[2].plot(x, y_dynamic_3, label='Dynamic Inputs')
ax[2].plot(x, y_not_dynamic_3, label='Fixed Inputs')

# Set labels and titles with increased font size
for i in range(3):
    ax[i].set_xlabel('Number of conversations regenerated', fontsize=18)
    ax[i].set_ylabel(f'Distinct-{i+1} Scores', fontsize=18)
    ax[i].tick_params(axis='both', which='major', labelsize=18)
    ax[i].set_ylim([0.3,1])
    ax[i].legend(fontsize=18)

# ax[0].set_title('Comparison Distinct-1 scores with/without dynamic pieces', fontsize=16)
# ax[1].set_title('Comparison Distinct-2 scores with/without dynamic pieces', fontsize=16)
# ax[2].set_title('Comparison Distinct-3 scores with/without dynamic pieces', fontsize=16)

# Add a general title
fig.suptitle('Comparison of Distinct-N Scores with and without Dynamic Inputs', fontsize=22)

# Adjust the layout
plt.tight_layout()  # rect to make space for the suptitle
plt.savefig("figs/distinct_n_score_comparison" + ".pdf")

# Show the plot
plt.show()